In [1]:
import pandas as pd
import warnings
warnings.simplefilter("ignore")

In [11]:
df = pd.read_csv('newark_flights.csv')
df.columns

Index(['Unnamed: 0', 'YEAR', 'MONTH', 'DAY', 'DAY_OF_WEEK', 'AIRLINE',
       'FLIGHT_NUMBER', 'TAIL_NUMBER', 'ORIGIN_AIRPORT', 'DESTINATION_AIRPORT',
       'SCHEDULED_DEPARTURE', 'DEPARTURE_TIME', 'DEPARTURE_DELAY', 'TAXI_OUT',
       'WHEELS_OFF', 'SCHEDULED_TIME', 'ELAPSED_TIME', 'AIR_TIME', 'DISTANCE',
       'WHEELS_ON', 'TAXI_IN', 'SCHEDULED_ARRIVAL', 'ARRIVAL_TIME',
       'ARRIVAL_DELAY', 'DIVERTED', 'CANCELLED', 'CANCELLATION_REASON',
       'AIR_SYSTEM_DELAY', 'SECURITY_DELAY', 'AIRLINE_DELAY',
       'LATE_AIRCRAFT_DELAY', 'WEATHER_DELAY'],
      dtype='object')

In [12]:
# Clean up null

df.drop('Unnamed: 0', axis=1, inplace=True)
df.drop(df[(df['ORIGIN_AIRPORT']=='EWR') & (df['DEPARTURE_TIME'].isna())].index, inplace=True)
df.drop(df[(df['DESTINATION_AIRPORT']=='EWR') & (df['ARRIVAL_TIME'].isna())].index, inplace=True)
df.drop(df[(df['ARRIVAL_DELAY'].isna())].index, inplace=True)
delay = list(df[df.columns[-6:]].columns)
for cols in delay:
    if df[cols].isna().any():
        df[cols].fillna(value=0, inplace=True)

In [13]:
# df.isna().sum()

In [14]:
# Clean up date and time columns

df['DATE'] = pd.to_datetime(df[['YEAR','MONTH', 'DAY']])
df["ARR_HOUR_SCH"] = df["SCHEDULED_ARRIVAL"].apply(lambda x: int(str(int(x))[0:2]) if len(str(int(x)))==4 else int(str(int(x))[0:1]))
df["ARR_MIN_SCH"]  = df["SCHEDULED_ARRIVAL"].apply(lambda x: int(str(int(x))[-2:]))
df["DEP_HOUR_SCH"] = df["SCHEDULED_DEPARTURE"].apply(lambda x: int(str(int(x))[0:2]) if len(str(int(x)))==4 else int(str(int(x))[0:1]))
df["DEP_MIN_SCH"]  = df["SCHEDULED_DEPARTURE"].apply(lambda x: int(str(int(x))[-2:]))
df = df.rename(columns={'ARR_HOUR_SCH':'HOUR', 'ARR_MIN_SCH':'MINUTE'})

df["SCHEDULED_ARRIVAL"] = pd.to_datetime(df[['YEAR','MONTH', 'DAY', 'HOUR', 'MINUTE']])
df["SCH_ARR_TEMP"] = pd.to_datetime(df[['YEAR','MONTH', 'DAY', 'HOUR']])
df.drop(['HOUR','MINUTE'], axis=1, inplace=True)
df = df.rename(columns={'DEP_HOUR_SCH':'HOUR', 'DEP_MIN_SCH':'MINUTE'})
df["SCHEDULED_DEPARTURE"] = pd.to_datetime(df[['YEAR','MONTH', 'DAY', 'HOUR', 'MINUTE']])
df["SCH_DEP_TEMP"] = pd.to_datetime(df[['YEAR','MONTH', 'DAY', 'HOUR']])

df["NYC_TIME_TEMP"] = df.apply(lambda row: row['SCH_ARR_TEMP'] if row['DESTINATION_AIRPORT'] == 'EWR' else row['SCH_DEP_TEMP'], axis=1)
df.drop(['HOUR','MINUTE', 'SCH_ARR_TEMP', 'SCH_DEP_TEMP'], axis=1, inplace=True)

In [15]:
# Import weather database

weather = pd.DataFrame(pd.read_csv('newark_weather.csv'))
weather.drop('Unnamed: 0', axis=1, inplace=True)
weather['temperature'] = weather['temperature'].apply(lambda row: round(row,1))
weather['nyc_time'] = pd.to_datetime(weather['nyc_time'])
weather = weather.rename(columns={'icon':'weather'})
weather = weather.set_index('nyc_time')
weather = weather[weather.columns[:-4]]
weather.head()

,weather,precipIntensity,temperature,windSpeed,visibility
nyc_time,,,,,
2015-01-01 00:00:00,clear,0.0,-4.4,1.66,9.997
2015-01-01 01:00:00,clear,0.0,-4.4,1.91,9.997
2015-01-01 02:00:00,clear,0.0,-4.4,2.38,9.997
2015-01-01 03:00:00,clear,0.0,-5.0,1.85,9.997
2015-01-01 04:00:00,clear,0.0,-4.4,2.77,9.997


In [16]:
# Join main data with weather database

df = df.join(weather, on='NYC_TIME_TEMP')
df.drop('NYC_TIME_TEMP', axis=1, inplace=True)

In [17]:
# Import plane registration database

plane_reg = pd.DataFrame(pd.read_csv('newark_plane_reg.csv'))
plane_reg.drop('Unnamed: 0', axis=1, inplace=True)
plane_reg = plane_reg.set_index('N-Number')
plane_reg.head()

,Reg_year,Reg_month
N-Number,,
N438WN,2003,7
N68061,2002,3
N914UY,2014,11
N446UA,1998,7
N18120,2005,2


In [18]:
# Join main data with plane registration database

df = df.join(plane_reg, on='TAIL_NUMBER')
df.drop(df[(df['Reg_year'].isna())].index, inplace=True)
df.head()

,YEAR,MONTH,DAY,DAY_OF_WEEK,AIRLINE,FLIGHT_NUMBER,TAIL_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,...,LATE_AIRCRAFT_DELAY,WEATHER_DELAY,DATE,weather,precipIntensity,temperature,windSpeed,visibility,Reg_year,Reg_month
0,2015,1,1,4,UA,1528,N76519,SJU,EWR,2015-01-01 01:54:00,...,0.0,0.0,2015-01-01,clear,0.0,-4.4,0.00,9.997,2010.0,1.0
1,2015,1,1,4,B6,1990,N597JB,SJU,EWR,2015-01-01 02:06:00,...,0.0,0.0,2015-01-01,clear,0.0,-4.4,0.00,9.997,2004.0,11.0
2,2015,1,1,4,UA,1162,N37293,BQN,EWR,2015-01-01 02:59:00,...,0.0,0.0,2015-01-01,clear,0.0,-4.4,0.00,9.997,2005.0,7.0
3,2015,1,1,4,EV,4160,N11150,JAX,EWR,2015-01-01 05:40:00,...,0.0,0.0,2015-01-01,clear,0.0,-5.6,1.46,9.997,2003.0,10.0
4,2015,1,1,4,EV,4646,N29917,CHS,EWR,2015-01-01 05:45:00,...,0.0,0.0,2015-01-01,clear,0.0,-5.6,1.46,9.997,2001.0,4.0


In [19]:
df.dtypes

YEAR                            int64
MONTH                           int64
DAY                             int64
DAY_OF_WEEK                     int64
AIRLINE                        object
FLIGHT_NUMBER                   int64
TAIL_NUMBER                    object
ORIGIN_AIRPORT                 object
DESTINATION_AIRPORT            object
SCHEDULED_DEPARTURE    datetime64[ns]
DEPARTURE_TIME                float64
DEPARTURE_DELAY               float64
TAXI_OUT                      float64
WHEELS_OFF                    float64
SCHEDULED_TIME                float64
ELAPSED_TIME                  float64
AIR_TIME                      float64
DISTANCE                        int64
WHEELS_ON                     float64
TAXI_IN                       float64
SCHEDULED_ARRIVAL      datetime64[ns]
ARRIVAL_TIME                  float64
ARRIVAL_DELAY                 float64
DIVERTED                        int64
CANCELLED                       int64
CANCELLATION_REASON             int64
AIR_SYSTEM_D